# TM(01) Mode of Step-Index Fiber - Full Vector Solution

This example demonstrates how to calculate the TM(01) mode of a step-index fiber. The magnetic field for this mode is azimuthally symmetric, meaning that both Hx and Hy have the same magnitude, which makes it an excellent test case for the full-vector modesolver. The example also compares the numerically computed mode with the exact analytical solution. Although the field profiles are nearly identical, there is a noticeable discrepancy between the numerical and exact eigenvalues. This difference is attributed to the geometric inaccuracy introduced by representing the curved core–cladding boundary using a staircase approximation.

In [ ]:
# Enable automatic reloading of modules (IPython only)
%reload_ext autoreload
%autoreload 2 

# load required modules
import modesolver as mode
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import newton   # Needed to find zero of function
from scipy.special import jv, kv         # Bessel J and K


In [ ]:
# Refractive indices:
nco = 2.5           # core index
ncl = 1.5           # cladding index

# Geometry
r = 0.30            # core radius (um)
side = 0.2

# Grid size:
dx = dy = 0.002     # grid size (um)

# Build waveguide mesh (eps)
x,y,xc,yc,nx,ny,eps = mode.fibermesh([nco, ncl],[r,r+side],dx,dy)

In [ ]:
# Stretch mesh on NORTH and EAST boundaries, to enlarge computational region
x,y,xc,yc,dx,dy = mode.stretchmesh(x,y,[96,0,96,0],[4,1,4,1])

In [ ]:
wavelength = 1      # vacuum wavelength
nmodes = 1          # number of modes to compute

# Find TM(01) mode:
neff, _, _, _, Hx, Hy, Hz = mode.wgmodes(wavelength,nco,nmodes,dx,dy,eps=eps,boundary='0A0S')
print(f"neff (finite difference) = {neff:.6f}")

hmax = np.max(np.abs(Hx))
Hx /= hmax
Hy /= hmax

In [ ]:
V = 2 * np.pi * r / wavelength * np.sqrt(nco**2 - ncl**2)

def func(U):
    term1 = nco**2 * jv(1, U) / (U * jv(0, U))
    W = np.sqrt(V**2 - U**2)
    term2 = ncl**2 * kv(1, W) / (W * kv(0, W))
    return term1 + term2

# Solve for U:
U = newton(func, x0 = 3.2)

W = np.sqrt(V**2 - U**2)

neff0 = np.sqrt(nco**2 - (U / (2 * np.pi * r / wavelength))**2)

print(f"neff (exact solution)    = {neff0:.6f}")

In [ ]:
# Now calculate the EXACT fields for the TM(01) mode:

X, Y = np.meshgrid(x, y, indexing='xy')

rho = np.sqrt(X**2 + Y**2)
sinphi = Y / rho
cosphi = X / rho

# Clean up divide-by-zero at the origin
sinphi = np.nan_to_num(sinphi)
cosphi = np.nan_to_num(cosphi)

Hx0 = np.zeros_like(rho)
Hy0 = np.zeros_like(rho)

# Masks based on radial position
mask_core = rho < r
mask_clad = ~mask_core

# Core region (rho < r)
Hx0[mask_core] = -sinphi[mask_core] * (
    jv(1, U * rho[mask_core] / r) / jv(1, U)
)
Hy0[mask_core] =  cosphi[mask_core] * (
    jv(1, U * rho[mask_core] / r) / jv(1, U)
)

# Cladding region (rho >= r)
Hx0[mask_clad] = -sinphi[mask_clad] * (
    kv(1, W * rho[mask_clad] / r) / kv(1, W)
)
Hy0[mask_clad] =  cosphi[mask_clad] * (
    kv(1, W * rho[mask_clad] / r) / kv(1, W)
)

# Normalize to max absolute value of Hx0
hmax = np.max(np.abs(Hx0))
Hx0 /= hmax
Hy0 /= hmax

In [ ]:
# Plot fields
levels = np.arange(-45,0,3)
fig, axes = plt.subplots(2, 2, figsize=(8,8))
axHx, axHy, axHx0, axHy0 = axes.ravel()

axHx.pcolormesh(x,y,abs(Hx),cmap='jet',vmax=1)
axHx.contour(x,y,20*np.log10(abs(Hx)),levels,colors='white',negative_linestyles='solid', linewidths=1)
axHx.set_aspect('equal')
axHx.axis([min(x), max(x), min(y), max(y)])
axHx.set_xlabel('x')
axHx.set_ylabel('y')
axHx.set_title(f'Hx (finite difference)')

axHy.pcolormesh(x,y,abs(Hy),cmap='jet',vmax=1)
axHy.contour(x,y,20*np.log10(abs(Hy)),levels,colors='white',negative_linestyles='solid', linewidths=1)
axHy.set_aspect('equal')
axHy.axis([min(x), max(x), min(y), max(y)])
axHy.set_xlabel('x')
axHy.set_ylabel('y')
axHy.set_title(f'Hy (finite difference)')

axHx0.pcolormesh(x,y,abs(Hx0),cmap='jet',vmax=1)
axHx0.contour(x,y,20*np.log10(abs(Hx0)),levels,colors='white',negative_linestyles='solid', linewidths=1)
axHx0.set_aspect('equal')
axHx0.axis([min(x), max(x), min(y), max(y)])
axHx0.set_xlabel('x')
axHx0.set_ylabel('y')
axHx0.set_title(f'Hx (exact)')

axHy0.pcolormesh(x,y,abs(Hy0),cmap='jet',vmax=1)
axHy0.contour(x,y,20*np.log10(abs(Hy0)),levels,colors='white',negative_linestyles='solid', linewidths=1)
axHy0.set_aspect('equal')
axHy0.axis([min(x), max(x), min(y), max(y)])
axHy0.set_xlabel('x')
axHy0.set_ylabel('y')
axHy0.set_title(f'Hy (exact)')

plt.show()